# Preparación de datos
## Importar bibliotecas

In [1]:
import os
import polars as pl
import polars.selectors as cs
import pandas as pd
from datetime import date, time
import numpy as np
import pyarrow.parquet as pq

In [2]:
# from os import listdir
# from os.path import isfile, join

In [2]:


# Ruta al directorio que contiene los archivos Parquet
directorio = "..\\datasets\\raw\\"

# Obtener la lista de archivos en el directorio
archivos_parquet = [f for f in os.listdir(directorio) if os.path.isfile(os.path.join(directorio, f)) and f.endswith('.parquet')]

# Crear un diccionario para almacenar los DataFrames de Polars
diccionario_dataframes = {}

# Iterar sobre los archivos Parquet
for archivo in archivos_parquet:
    # Verificar si el archivo contiene "yellow", ".parquet" y "2023" en su nombre
    if 'yellow' in archivo and '.parquet' in archivo and '2022' in archivo:
        # Obtener el mes del archivo
        mes = archivo.split('_')[2].split('-')[1].split('.')[0]  # Extraer el mes del nombre del archivo
        
        # Construir la ruta completa al archivo
        ruta_archivo = os.path.join(directorio, archivo)
        
        # Leer el archivo Parquet en un DataFrame de Polars
        df = pl.read_parquet(ruta_archivo)
        
        # Agregar el DataFrame al diccionario utilizando el nombre del archivo como clave
        diccionario_dataframes[mes] = df

# Ahora el diccionario 'diccionario_dataframes' contiene un DataFrame por mes para el año 2023
# de los archivos Parquet que contienen "yellow" en su nombre.


In [3]:
# Iterar sobre el diccionario de dataframes
for nombre, df in diccionario_dataframes.items():
    print(f"Información del DataFrame '{nombre}':")
    print("Cantidad de registros :", len(df), "Cantidad de las Columnas :", len(df.columns))
    print("Schema")
    #print(df.schema)
    print("\n")

Información del DataFrame '01':
Cantidad de registros : 2463931 Cantidad de las Columnas : 19
Schema


Información del DataFrame '02':
Cantidad de registros : 2979431 Cantidad de las Columnas : 19
Schema


Información del DataFrame '03':
Cantidad de registros : 3627882 Cantidad de las Columnas : 19
Schema


Información del DataFrame '04':
Cantidad de registros : 3599920 Cantidad de las Columnas : 19
Schema


Información del DataFrame '05':
Cantidad de registros : 3588295 Cantidad de las Columnas : 19
Schema


Información del DataFrame '06':
Cantidad de registros : 3558124 Cantidad de las Columnas : 19
Schema


Información del DataFrame '07':
Cantidad de registros : 3174394 Cantidad de las Columnas : 19
Schema


Información del DataFrame '08':
Cantidad de registros : 3152677 Cantidad de las Columnas : 19
Schema


Información del DataFrame '09':
Cantidad de registros : 3183767 Cantidad de las Columnas : 19
Schema


Información del DataFrame '10':
Cantidad de registros : 3675411 Cantidad 

In [4]:
# Supongamos que diccionario_dataframes es tu diccionario que contiene los DataFrames
# Reemplaza diccionario_dataframes con el nombre de tu diccionario
"""
# Tomar el primer DataFrame del diccionario
primer_df = next(iter(diccionario_dataframes.values()))

# Definir un nuevo esquema con los tipos de datos y nombres de columna deseados
nuevo_esquema = {
    "VendorID": pl.Int32,
    "tpep_pickup_datetime": pl.Datetime(time_unit='ns', time_zone=None),
    "tpep_dropoff_datetime": pl.Datetime(time_unit='ns', time_zone=None),
    "passenger_count": pl.Int64,
    "trip_distance": pl.Float64,
    "RatecodeID": pl.Int64,
    "store_and_fwd_flag": pl.String,
    "PULocationID": pl.Int32,
    "DOLocationID": pl.Int32,
    "payment_type": pl.Int64,
    "fare_amount": pl.Float64,
    "extra": pl.Float64,
    "mta_tax": pl.Float64,
    "tip_amount": pl.Float64,
    "tolls_amount": pl.Float64,
    "improvement_surcharge": pl.Float64,
    "total_amount": pl.Float64,
    "congestion_surcharge": pl.Float64,
    "airport_fee": pl.Float64,  # He cambiado el nombre de la columna a "airport_fee"
}

# Renombrar las columnas y cambiar los tipos de datos del DataFrame
df = primer_df.select([
    pl.col(col).cast(dtype).alias(col) for col, dtype in nuevo_esquema.items()
])

# Cambiar el nombre de la columna "airport_fee" a "Airport_fee"
df = df.with_columns(primer_df['airport_fee'].alias('Airport_fee'))
df = df.drop('airport_fee')
# Reemplazar el DataFrame original en el diccionario con el DataFrame modificado
nombre_clave = next(iter(diccionario_dataframes))  # Obtener la clave del primer DataFrame
diccionario_dataframes[nombre_clave] = df

# Verificar el resultado
print(diccionario_dataframes[nombre_clave].schema)

"""

OrderedDict([('VendorID', Int32), ('tpep_pickup_datetime', Datetime(time_unit='ns', time_zone=None)), ('tpep_dropoff_datetime', Datetime(time_unit='ns', time_zone=None)), ('passenger_count', Int64), ('trip_distance', Float64), ('RatecodeID', Int64), ('store_and_fwd_flag', String), ('PULocationID', Int32), ('DOLocationID', Int32), ('payment_type', Int64), ('fare_amount', Float64), ('extra', Float64), ('mta_tax', Float64), ('tip_amount', Float64), ('tolls_amount', Float64), ('improvement_surcharge', Float64), ('total_amount', Float64), ('congestion_surcharge', Float64), ('Airport_fee', Float64)])


In [6]:
# Iterar sobre el diccionario de dataframes
# for nombre, df in diccionario_dataframes.items():
#     print(f"Información del DataFrame '{nombre}':")
#     print("Cantidad de registros :", len(df), "Cantidad de las Columnas :", len(df.columns))
#     print("Schema")
#     print(df.schema)
#     print("\n")

In [4]:
# Ahora que todos los DataFrames tienen las mismas columnas, puedes proceder a concatenarlos.
dataframes = list(diccionario_dataframes.values())

# Concatenar verticalmente todos los DataFrames
df_concatenado = pl.concat(dataframes)

In [5]:
df_concatenado.shape

(39656098, 19)

In [6]:
df_concatenado.schema

OrderedDict([('VendorID', Int64),
             ('tpep_pickup_datetime',
              Datetime(time_unit='ns', time_zone=None)),
             ('tpep_dropoff_datetime',
              Datetime(time_unit='ns', time_zone=None)),
             ('passenger_count', Float64),
             ('trip_distance', Float64),
             ('RatecodeID', Float64),
             ('store_and_fwd_flag', String),
             ('PULocationID', Int64),
             ('DOLocationID', Int64),
             ('payment_type', Int64),
             ('fare_amount', Float64),
             ('extra', Float64),
             ('mta_tax', Float64),
             ('tip_amount', Float64),
             ('tolls_amount', Float64),
             ('improvement_surcharge', Float64),
             ('total_amount', Float64),
             ('congestion_surcharge', Float64),
             ('airport_fee', Float64)])

In [10]:
df_concatenado.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,"""N""",142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,"""N""",236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,"""N""",166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,"""N""",114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,"""N""",68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0


In [12]:
# Obtener el recuento de valores nulos como un objeto lazy
lazy_result = df_concatenado.lazy().null_count()

# Ejecutar la operación lazy y obtener el resultado
result = lazy_result.collect()

# Mostrar el resultado
result

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,1368303,0,1368303,1368303,0,0,0,0,0,0,0,0,0,0,1368303,1368303


In [14]:
# Obtener solo las columnas numéricas 
numeric_columns = [col_name for col_name in df_concatenado.columns if df_concatenado[col_name].dtype.is_numeric()]

# Convertir la selección de columnas en una operación lazy
lazy_select = df_concatenado.select(numeric_columns)

# Calcular estadísticas descriptivas solo para las columnas numéricas de manera lazy
lazy_stats = lazy_select.describe()

lazy_stats

statistic,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",3.9656098e7,3.8287795e7,3.9656098e7,3.8287795e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.8287795e7,3.8287795e7
"""null_count""",0.0,1.368303e6,0.0,1.368303e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.368303e6,1.368303e6
"""mean""",1.721802,1.401149,5.959399,1.424172,164.865988,162.575201,1.18955,10.362822,1.007532,0.488806,7.234908,0.536283,0.316124,21.671268,2.281437,0.097419
"""std""",0.480527,0.962894,599.190714,5.794343,65.310815,70.231459,0.519041,22328.299433,1.262564,0.093804,22328.083995,2.043351,0.129332,96.373602,0.751943,0.338539
"""min""",1.0,0.0,0.0,1.0,1.0,1.0,0.0,-1.3339e8,-22.18,-0.55,-410.0,-99.99,-1.0,-2567.8,-2.5,-1.25
"""25%""",1.0,1.0,1.1,1.0,132.0,113.0,1.0,7.0,0.0,0.5,0.93,0.0,0.3,12.3,2.5,0.0
"""50%""",2.0,1.0,1.9,1.0,162.0,162.0,1.0,10.5,0.5,0.5,2.16,0.0,0.3,15.96,2.5,0.0
"""75%""",2.0,1.0,3.56,1.0,234.0,234.0,1.0,16.5,2.5,0.5,3.36,0.0,0.3,23.16,2.5,0.0
"""max""",6.0,9.0,389678.46,99.0,265.0,265.0,5.0,401092.32,33.5,25.48,1.3339e8,911.87,1.0,401095.62,2.75,1.25


In [16]:
df_nyc = df_concatenado.drop_nulls()

In [17]:
df_nyc.shape

(38287795, 19)

## General Preprocessing
### Preprocesamiento Variables `Temporales` 

In [19]:
# Crear un contexto SQL y registrar el DataFrame como una tabla
sql = pl.SQLContext(frames={"df_nyc": df_nyc})

result = sql.execute(
    "SELECT * FROM df_nyc LIMIT 15"
)


# Convertir el resultado a un DataFrame de Pandas para mostrarlo
result_df = result.collect().to_pandas()

# Mostrar el DataFrame resultante
result_df.head(15)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.00,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.00,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.00,0.3,30.30,2.5,0.0
5,1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.30,1.0,N,138,161,1,33.0,3.0,0.5,13.00,6.55,0.3,56.35,2.5,0.0
6,2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.20,0.00,0.3,26.00,2.5,0.0
7,2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.00,0.00,0.3,12.80,2.5,0.0
8,2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.00,0.3,18.05,2.5,0.0
9,2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.00,0.00,0.3,8.80,2.5,0.0


In [20]:
# Crear un contexto SQL y registrar el DataFrame como una tabla
sql = pl.SQLContext(frames={"df_nyc": df_nyc})

result = sql.execute(
    "SELECT DISTINCT tpep_pickup_datetime FROM df_nyc LIMIT 15"
)


# Convertir el resultado a un DataFrame de Pandas para mostrarlo
result_df = result.collect().to_pandas()

# Mostrar el DataFrame resultante
result_df

,tpep_pickup_datetime
0,2022-01-01 00:35:40
1,2022-01-01 00:33:43
2,2022-01-01 00:53:21
3,2022-01-01 00:25:21
4,2022-01-01 00:36:48
5,2022-01-01 00:40:15
6,2022-01-01 00:20:50
7,2022-01-01 00:13:04
8,2022-01-01 00:30:02
9,2022-01-01 00:48:52


In [22]:
from datetime import datetime

In [23]:
# Convertir las columnas de fechas a tipo DateTime en Polars
df_nyc = df_nyc.with_columns(
    pl.col('tpep_pickup_datetime').dt.date().alias('tpep_pickup_date'),
    pl.col('tpep_dropoff_datetime').dt.date().alias('tpep_dropoff_date')
)

# Calcular la duración del viaje en segundos
df_nyc = df_nyc.with_columns(
    (pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_seconds().alias('viaje_segundos')
)
# Eliminar las columnas originales de fecha y hora
df_nyc = df_nyc.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'])


In [24]:
# Eliminar las columnas originales de fecha y hora
df_nyc = df_nyc.drop(['store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee'])

In [25]:
df_nyc.shape

(38287795, 11)

In [26]:
# Crear un contexto SQL y registrar el DataFrame como una tabla
sql = pl.SQLContext(frames={"df_nyc": df_nyc})

result = sql.execute(
    "SELECT * FROM df_nyc LIMIT 15"
)


# Convertir el resultado a un DataFrame de Pandas para mostrarlo
result_df = result.collect().to_pandas()

# Mostrar el DataFrame resultante
result_df.head(15)

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,total_amount,tpep_pickup_date,tpep_dropoff_date,viaje_segundos
0,1,2.0,3.80,1.0,142,236,1,21.95,2022-01-01,2022-01-01,1069
1,1,1.0,2.10,1.0,236,42,1,13.30,2022-01-01,2022-01-01,504
2,2,1.0,0.97,1.0,166,166,1,10.56,2022-01-01,2022-01-01,538
3,2,1.0,1.09,1.0,114,68,2,11.80,2022-01-01,2022-01-01,602
4,2,1.0,4.30,1.0,68,163,1,30.30,2022-01-01,2022-01-01,2252
5,1,1.0,10.30,1.0,138,161,1,56.35,2022-01-01,2022-01-01,1773
6,2,1.0,5.07,1.0,233,87,1,26.00,2022-01-01,2022-01-01,848
7,2,1.0,2.02,1.0,238,152,2,12.80,2022-01-01,2022-01-01,581
8,2,1.0,2.71,1.0,166,236,1,18.05,2022-01-01,2022-01-01,887
9,2,1.0,0.78,1.0,236,141,2,8.80,2022-01-01,2022-01-01,276


In [22]:
# Convertir el DataFrame de Polars a un objeto Arrow
arrow_table = df_nyc.to_arrow()


In [24]:

# Guardar el objeto Arrow en un archivo Arrow
#pq.write_table(arrow_table, 'temperature.arrow')
#pq.write_table(arrow_table, '../datasets/processed/df_yellow.arrow')
pq.write_table(arrow_table, '../datasets/processed/yellow.arrow', compression='zstd')

In [ ]:
#df_nyc.describe()
df_nyc.schema

In [ ]:
# Calcular la media de la columna 'tviaje_segundos'
mean_seconds = df_nyc['viaje_segundos'].mean()

# Calcular la mediana de la columna 'tviaje_segundos'
median_seconds = df_nyc['viaje_segundos'].median()

# Calcular la desviación estándar de la columna 'tviaje_segundos'
std_seconds = df_nyc['viaje_segundos'].std()

# Obtener el valor mínimo de la columna 'viaje_segundos'
min_seconds = df_nyc['viaje_segundos'].min()

# Obtener el valor máximo de la columna 'viaje_segundos'
max_seconds = df_nyc['viaje_segundos'].max()

# Imprimir los resultados
print("Minimo:", min_seconds)
print("Maximo:", max_seconds)
print("Media:", mean_seconds)
print("Mediana:", median_seconds)
print("Desviación estándar:", std_seconds)


In [ ]:
# Filtrar los registros con valores negativos en la columna 'viaje_segundos'
negativos = df_nyc.filter(pl.col('viaje_segundos') < 0.0)

# Obtener la cantidad de registros con valores negativos
cantidad_negativos = negativos.height
print("Cantidad de registros con valores negativos en 'viaje_segundos':", cantidad_negativos)



In [ ]:
# Eliminar los registros con valores negativos en la columna 'viaje_segundos'
#df_nyc = df_nyc.drop(pl.col('viaje_segundos') < 0.0)
# Filtrar las filas que cumplen la condición
df_yellow = df_nyc.filter(pl.col('viaje_segundos') >= 0)

In [ ]:
#df = pl.scan_ipc("temperature.arrow")
# create a SQL context, registering the frame as a table
#sql = pl.SQLContext(my_table=df)
# create a SQL query to execute
# Leer el archivo Arrow
arrow_table = pq.read_table('../datasets/processed/temperature.arrow')

# Convertir el objeto Arrow a un DataFrame de Polars
df_polars = pl.from_arrow(arrow_table)

In [ ]:
df_polars 

In [ ]:
import pandas as pd
import polars as pl
import pyarrow.parquet as pq
import pyarrow.dataset as ds

# Ejemplo de DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

# Convertir el DataFrame de Pandas a un DataFrame de Polars
df_polars = pl.from_pandas(df)

# Convertir el DataFrame de Polars a un objeto Arrow
arrow_table = df_polars.to_arrow()

# Guardar el objeto Arrow en un archivo Arrow
pq.write_table(arrow_table, 'data.arrow')

# Leer el archivo Arrow utilizando pl.scan_pyarrow_dataset()
dset = ds.dataset("data.arrow", format="ipc")  
df_loaded = (
    pl.scan_pyarrow_dataset(dset)
    .to_pandas()  # Convertir a DataFrame de Pandas para mostrarlo
)

# Mostrar el DataFrame cargado
print(df_loaded)



In [33]:
import polars as pl
import pyarrow.parquet as pq

# Leer el archivo Arrow
arrow_table = pq.read_table('../datasets/processed/temperature.arrow')

# Convertir el objeto Arrow a un DataFrame de Polars
df_polars = pl.from_arrow(arrow_table)

In [34]:
df_polars

event_date,temperature_2m
"datetime[ns, UTC]",f32
2021-01-01 00:00:00 UTC,3.696
2021-01-01 01:00:00 UTC,3.496
2021-01-01 02:00:00 UTC,2.946
2021-01-01 03:00:00 UTC,0.946
2021-01-01 04:00:00 UTC,0.096
…,…
2023-12-31 19:00:00 UTC,6.396
2023-12-31 20:00:00 UTC,6.296
2023-12-31 21:00:00 UTC,5.796


In [35]:
# Crear un contexto SQL y registrar el DataFrame como una tabla
sql = pl.SQLContext(frames={"df_polars": df_polars})

result = sql.execute(
    "SELECT * FROM df_polars WHERE temperature_2m < 10"
)



# Convertir el resultado a un DataFrame de Pandas para mostrarlo
result_df = result.collect().to_pandas()

# Mostrar el DataFrame resultante
result_df.head(15)


,event_date,temperature_2m
0,2021-01-01 00:00:00+00:00,3.696
1,2021-01-01 01:00:00+00:00,3.496
2,2021-01-01 02:00:00+00:00,2.946
3,2021-01-01 03:00:00+00:00,0.946
4,2021-01-01 04:00:00+00:00,0.096
5,2021-01-01 05:00:00+00:00,-0.704
6,2021-01-01 06:00:00+00:00,-1.254
7,2021-01-01 07:00:00+00:00,-1.654
8,2021-01-01 08:00:00+00:00,-2.104
9,2021-01-01 09:00:00+00:00,-2.404


In [30]:
import polars as pl

# Crear un contexto SQL y registrar el DataFrame como una tabla
sql = pl.SQLContext(frames={"df_polars": df_polars})

# Ejecutar la consulta SQL y seleccionar los primeros 15 registros
result = sql.execute("SELECT * FROM df_polars LIMIT 15")

# Convertir el LazyFrame a un DataFrame de Polars
result_df = pl.DataFrame(result.collect())

# Iterar sobre las filas del DataFrame
for row in result_df.rows():
    print(row)


(datetime.datetime(2021, 1, 1, 0, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), 3.69599986076355)
(datetime.datetime(2021, 1, 1, 1, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), 3.495999813079834)
(datetime.datetime(2021, 1, 1, 2, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), 2.94599986076355)
(datetime.datetime(2021, 1, 1, 3, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), 0.9459999799728394)
(datetime.datetime(2021, 1, 1, 4, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), 0.09600000083446503)
(datetime.datetime(2021, 1, 1, 5, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), -0.7039999961853027)
(datetime.datetime(2021, 1, 1, 6, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), -1.253999948501587)
(datetime.datetime(2021, 1, 1, 7, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), -1.653999924659729)
(datetime.datetime(2021, 1, 1, 8, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), -2.1040000915527344)
(datetime.datetime(2021, 1, 1, 9, 0, tzinfo=zoneinfo.ZoneInfo(key='UTC')), -2.4040000438690186)
(datetime.datetime(2021, 1, 1, 10, 0, tzinfo=zonein

In [31]:
type(result_df)

polars.dataframe.frame.DataFrame

In [32]:
result_df.head(15)

event_date,temperature_2m
"datetime[ns, UTC]",f32
2021-01-01 00:00:00 UTC,3.696
2021-01-01 01:00:00 UTC,3.496
2021-01-01 02:00:00 UTC,2.946
2021-01-01 03:00:00 UTC,0.946
2021-01-01 04:00:00 UTC,0.096
…,…
2021-01-01 10:00:00 UTC,-2.504
2021-01-01 11:00:00 UTC,-2.254
2021-01-01 12:00:00 UTC,-1.454
